In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import os
import csv
import pandas as pd
import numpy as np
import requests
import time
import json
from pandas import json_normalize
from scipy.stats import linregress
from pprint import pprint

In [2]:
# Set the API base URL 
json_url = f"https://data.austintexas.gov/resource/9t4d-g238.json"

# Create URL for GET requests
animal_data = requests.get(json_url).json()

# Convert the API to a DataFrame and display data
animal_df = pd.DataFrame.from_dict(json_normalize(animal_data))
animal_df.tail(5)

,animal_id,name,datetime,monthyear,date_of_birth,outcome_type,animal_type,sex_upon_outcome,age_upon_outcome,breed,color,outcome_subtype
995,A718620,*Toby Sue,2016-03-01T13:53:00.000,2016-03-01T13:53:00.000,2015-07-23T00:00:00.000,Adoption,Cat,Spayed Female,7 months,Domestic Shorthair Mix,Black/White,NaN
996,A705114,NaN,2015-06-13T15:47:00.000,2015-06-13T15:47:00.000,2015-06-11T00:00:00.000,Transfer,Dog,Intact Male,2 days,German Shepherd Mix,Brown,Partner
997,A800888,NaN,2019-08-01T15:27:00.000,2019-08-01T15:27:00.000,2019-07-03T00:00:00.000,Transfer,Dog,Intact Female,4 weeks,Cavalier Span,Black/White,Partner
998,A796517,Chiquis,2019-06-08T12:21:00.000,2019-06-08T12:21:00.000,2017-06-02T00:00:00.000,Adoption,Dog,Spayed Female,2 years,Chihuahua Shorthair Mix,Brown/White,NaN
999,A880162,NaN,2023-05-09T17:56:00.000,2023-05-09T17:56:00.000,2023-03-05T00:00:00.000,Adoption,Cat,Neutered Male,2 months,Domestic Shorthair,Black/White,NaN


In [3]:
## JSON results are limited and will not provide 2022 information.
## Switching from JSON to CSV.

In [15]:
# Path to collect CSV data 
csv_url = "https://github.com/SariStormdance/Project_1/blob/main/Resources/Animal_Outcome.csv?raw=true"

# Convert CSV to DataFrame
initial_df = pd.read_csv(csv_url)
initial_df.tail(5)

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
153437,A883539,*Pipsqueak,07/19/2023 05:18:00 PM,Jul 2023,02/24/2023,Adoption,NaN,Dog,Intact Female,4 months,Labrador Retriever Mix,Cream/White
153438,A884334,Noche,07/19/2023 05:14:00 PM,Jul 2023,04/01/2023,Adoption,NaN,Cat,Spayed Female,3 months,Domestic Shorthair,Black
153439,A885055,Sparky,07/19/2023 05:00:00 PM,Jul 2023,04/12/2023,Adoption,NaN,Cat,Neutered Male,3 months,Siamese,Flame Point
153440,A884031,Bowzer,07/19/2023 05:26:00 PM,Jul 2023,06/27/2019,Adoption,NaN,Dog,Neutered Male,4 years,Labrador Retriever Mix,Cream/Yellow
153441,A883741,*Fiddle,07/19/2023 06:57:00 PM,Jul 2023,06/22/2022,Transfer,Partner,Dog,Intact Male,1 year,Australian Cattle Dog Mix,Tan/White


In [16]:
# Creating a DataFrame copy to reduce to essential columns
reduced_columns_df= initial_df[["Animal ID", "MonthYear", "Outcome Type", "Animal Type", "Breed"]].copy()
reduced_columns_df.tail(5)

,Animal ID,MonthYear,Outcome Type,Animal Type,Breed
153437,A883539,Jul 2023,Adoption,Dog,Labrador Retriever Mix
153438,A884334,Jul 2023,Adoption,Cat,Domestic Shorthair
153439,A885055,Jul 2023,Adoption,Cat,Siamese
153440,A884031,Jul 2023,Adoption,Dog,Labrador Retriever Mix
153441,A883741,Jul 2023,Transfer,Dog,Australian Cattle Dog Mix


In [17]:
# Rename column and split a DataFrame column
reduced_columns_df.rename(columns = {"MonthYear":"Month Year"}, inplace = True)
reduced_columns_df[["Month","Year"]] = reduced_columns_df["Month Year"].str.split(" ",expand=True)
reduced_columns_df.tail(5)

,Animal ID,Month Year,Outcome Type,Animal Type,Breed,Month,Year
153437,A883539,Jul 2023,Adoption,Dog,Labrador Retriever Mix,Jul,2023
153438,A884334,Jul 2023,Adoption,Cat,Domestic Shorthair,Jul,2023
153439,A885055,Jul 2023,Adoption,Cat,Siamese,Jul,2023
153440,A884031,Jul 2023,Adoption,Dog,Labrador Retriever Mix,Jul,2023
153441,A883741,Jul 2023,Transfer,Dog,Australian Cattle Dog Mix,Jul,2023


In [20]:
# Cleaning and rearranging the data
updated_columns_df = reduced_columns_df[["Animal ID", "Month", "Year", "Outcome Type", "Animal Type", "Breed"]].copy()
updated_columns_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
153437,A883539,Jul,2023,Adoption,Dog,Labrador Retriever Mix
153438,A884334,Jul,2023,Adoption,Cat,Domestic Shorthair
153439,A885055,Jul,2023,Adoption,Cat,Siamese
153440,A884031,Jul,2023,Adoption,Dog,Labrador Retriever Mix
153441,A883741,Jul,2023,Transfer,Dog,Australian Cattle Dog Mix


In [21]:
# Data check: Displaying number of records per year
updated_columns_df["Year"].value_counts()

2019    19775
2014    18709
2015    18506
2016    17670
2017    17657
2018    16741
2021    11966
2022    11880
2020     9777
2023     6256
2013     4505
Name: Year, dtype: int64

In [30]:
# Narrowing dataset to 2022 timeframe
defined_year_df = updated_columns_df[updated_columns_df["Year"] == "2022"]
defined_year_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
153073,A857909,Jun,2022,Adoption,Dog,Pit Bull Mix
153108,A853013,Apr,2022,Adoption,Cat,Domestic Shorthair
153197,A862948,Aug,2022,Transfer,Cat,Domestic Shorthair Mix
153355,A850898,Feb,2022,Adoption,Dog,Siberian Husky Mix
153422,A869924,Dec,2022,Transfer,Cat,Domestic Shorthair


In [31]:
# Resetting index for tracking purposes
index_reset_df = defined_year_df.reset_index(drop=True)
index_reset_df.tail(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
11875,A857909,Jun,2022,Adoption,Dog,Pit Bull Mix
11876,A853013,Apr,2022,Adoption,Cat,Domestic Shorthair
11877,A862948,Aug,2022,Transfer,Cat,Domestic Shorthair Mix
11878,A850898,Feb,2022,Adoption,Dog,Siberian Husky Mix
11879,A869924,Dec,2022,Transfer,Cat,Domestic Shorthair


In [43]:
#Display total animal counts
index_reset_df["Animal Type"].value_counts()

Dog          6412
Cat          4766
Other         592
Bird          109
Livestock       1
Name: Animal Type, dtype: int64

In [44]:
# Create separate dataframe for cat outcomes
Cat = ["Cat"] 
cat_outcomes_df = index_reset_df[index_reset_df["Animal Type"].isin(Cat)]
cat_outcomes_df.head(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
0,A860161,Jul,2022,Adoption,Cat,Domestic Shorthair
2,A848454,Jan,2022,Adoption,Cat,Domestic Shorthair Mix
3,A822928,Jan,2022,Adoption,Cat,Domestic Shorthair
5,A852822,Mar,2022,Adoption,Cat,Domestic Shorthair
13,A856357,Jun,2022,Adoption,Cat,Domestic Shorthair


In [45]:
# Data check: display monthly cat totals
cat_outcomes_df["Month"].value_counts()

Aug    587
Jun    581
Jul    536
May    503
Sep    430
Jan    375
Dec    368
Nov    335
Oct    316
Apr    296
Mar    228
Feb    211
Name: Month, dtype: int64

In [46]:
# Data check: display cat outcome types
cat_outcomes_df["Outcome Type"].value_counts()

Adoption           3194
Transfer           1177
Return to Owner     152
Euthanasia          142
Died                 54
Rto-Adopt            31
Disposal             12
Missing               3
Relocate              1
Name: Outcome Type, dtype: int64

In [47]:
# Create separate dataframe for dog outcomes
Dog = ["Dog"] 
dog_outcomes_df = index_reset_df[index_reset_df["Animal Type"].isin(Dog)]
dog_outcomes_df.head(5)

,Animal ID,Month,Year,Outcome Type,Animal Type,Breed
1,A595061,Aug,2022,Return to Owner,Dog,Labrador Retriever Mix
4,A856853,May,2022,Adoption,Dog,Black Mouth Cur Mix
6,A851472,Feb,2022,Transfer,Dog,Cairn Terrier Mix
7,A870443,Dec,2022,Adoption,Dog,Jack Russell Terrier/Chihuahua Shorthair
8,A851434,Feb,2022,Transfer,Dog,Chihuahua Shorthair


In [48]:
# Data check: display monthly dog totals
dog_outcomes_df["Month"].value_counts()

Aug    641
Apr    603
Jan    600
Jul    583
Sep    562
Jun    551
Mar    539
Feb    536
May    519
Oct    441
Dec    435
Nov    402
Name: Month, dtype: int64

In [49]:
# Data check: display dog outcome types
dog_outcomes_df["Outcome Type"].value_counts()

Adoption           3410
Transfer           1782
Return to Owner     951
Rto-Adopt           123
Euthanasia           97
Died                 34
Disposal             13
Stolen                2
Name: Outcome Type, dtype: int64